In [2]:
!pip install qrcode[pil] reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 10.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00


In [22]:
import qrcode
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import mm
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.colors import black
import os

# Конфигурация
COLS = 5  # Количество столбцов
QR_SIZE = 40*mm  # Размер QR-кода
TEXT_HEIGHT = 1*mm  # Высота области подписи
MARGIN = 3*mm  # Поля страницы
SPACING = 1*mm  # Расстояние между элементами

def generate_qr_with_text(text, filename):
    # Генерация QR-кода
    qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=2,
    )
    qr.add_data(text)
    qr.make(fit=True)
    img = qr.make_image(fill_color="black", back_color="white")
    img.save(filename)

def create_qr_sheet(output_filename, items):
    c = canvas.Canvas(output_filename, pagesize=A4)
    page_width, page_height = A4
    
    # Расчет доступной области
    content_width = page_width - 2*MARGIN
    content_height = page_height - 2*MARGIN
    
    # Расчет количества строк, которые поместятся на странице
    cell_height = QR_SIZE + TEXT_HEIGHT + 5*mm
    ROWS = int((content_height + SPACING) / (cell_height + SPACING))
    
    # Расчет размеров ячейки
    cell_width = (content_width - (COLS-1)*SPACING) / COLS
    
    # Установка шрифта
    font_path = "./arial.ttf"
    try:
        pdfmetrics.registerFont(TTFont('Arial', font_path))
        font_name = 'Arial'
    except:
        print(f"Не удалось загрузить шрифт по пути: {font_path}")
        font_name = 'Helvetica'
    
    for idx, item in enumerate(items):
        if idx > 0 and idx % (COLS * ROWS) == 0:
            c.showPage()  # Новая страница после заполнения
            
        # Позиция текущей ячейки
        page_idx = idx % (COLS * ROWS)
        col = page_idx % COLS
        row = page_idx // COLS  # Прямой порядок строк (сверху вниз)
        
        x = MARGIN + col * (cell_width + SPACING)
        y = page_height - MARGIN - (row + 1) * cell_height  # Расчет Y от верхнего края
        
        # Генерация QR
        qr_filename = f"temp_qr_{idx}.png"
        generate_qr_with_text(item, qr_filename)
        
        # Размещение QR-кода (по центру ячейки по горизонтали)
        qr_x = x + (cell_width - QR_SIZE) / 2
        qr_y = y + TEXT_HEIGHT + 5*mm  # QR над текстом с отступом
        c.drawImage(qr_filename, qr_x, qr_y, QR_SIZE, QR_SIZE)
        
        # Размещение текста под QR-кодом
        c.setFont(font_name, 10)
        c.setFillColor(black)
        
        # Проверяем ширину текста
        text_width = c.stringWidth(item, font_name, 10)
        max_width = cell_width - 4*mm
        
        if text_width > max_width:
            # Разбиваем текст на две строки
            mid = len(item) // 2
            # Ищем пробел для разбивки
            space_pos = item.find(' ', mid)
            if space_pos > 0:
                part1 = item[:space_pos]
                part2 = item[space_pos+1:]
            else:
                part1 = item[:mid]
                part2 = item[mid:]
            
            # Первая строка текста
            part1_width = c.stringWidth(part1, font_name, 10)
            text_x = x + (cell_width - part1_width) / 2
            text_y = y + 2*mm  # Отступ от нижнего края ячейки
            c.drawString(text_x, text_y, part1)
            
            # Вторая строка текста
            part2_width = c.stringWidth(part2, font_name, 10)
            text_x = x + (cell_width - part2_width) / 2
            text_y = y - 4*mm  # Отступ между строками
            c.drawString(text_x, text_y, part2)
        else:
            # Текст помещается в одну строку
            text_x = x + (cell_width - text_width) / 2
            text_y = y + 2*mm  # Отступ от нижнего края ячейки
            c.drawString(text_x, text_y, item)
        
        os.remove(qr_filename)  # Удаление временного файла
    
    c.save()

# Пример использования
# items = [f"Shelf-{i:03d}" for i in range(1, 101)]  # Генерация 100 кодов
items = [
    "С1П1С1", "С1П1С2", "С1П2С1", "С1П2С2", "С1П3С1",
    "6", "7", "8", "9", "10",
    "6", "7", "8", "9", "10",
    "6", "7", "8", "9", "10",
    "6", "7", "8", "9", "10",
    "6", "7", "8", "9", "10"
]
create_qr_sheet("warehouse_qrs.pdf", items)

Предупреждение: не найден шрифт с поддержкой кириллицы. Русский текст может отображаться некорректно.
